In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import random

# Simplified versions for demo

class SimplifiedESGEnvironment:
    """A simplified version of your ESG environment for demonstration purposes"""

    def __init__(self, company_data):
        self.company_data = company_data
        self.reset()

    def reset(self):
        self.current_state = {
            'esg_score': self.company_data['initial_scores']['overall'],
            'env_score': self.company_data['initial_scores']['env'],
            'social_score': self.company_data['initial_scores']['social'],
            'gov_score': self.company_data['initial_scores']['gov'],
            'financial_impact': 0,
            'step': 0,
            'actions_taken': []
        }
        return self.current_state

    def step(self, action_id):
        """Take an action and update the state"""
        action = self.company_data['available_actions'][action_id]

        # Apply the action effects (simplified model)
        pillar = action['pillar']
        improvement = action['base_improvement']

        # Add some variability to make it interesting
        variability = random.uniform(0.8, 1.2)
        improvement *= variability

        # Apply diminishing returns if action is repeated
        repeated = False
        for past_action in self.current_state['actions_taken']:
            if past_action['id'] == action_id:
                repeated = True
                improvement *= 0.5  # 50% effectiveness for repeated actions

        # Update the state
        if pillar == 'environmental':
            self.current_state['env_score'] += improvement
        elif pillar == 'social':
            self.current_state['social_score'] += improvement
        elif pillar == 'governance':
            self.current_state['gov_score'] += improvement
        elif pillar == 'all':
            self.current_state['env_score'] += improvement * 0.33
            self.current_state['social_score'] += improvement * 0.33
            self.current_state['gov_score'] += improvement * 0.34

        # Update overall ESG score (simplified)
        self.current_state['esg_score'] = (
            self.current_state['env_score'] * 0.33 +
            self.current_state['social_score'] * 0.33 +
            self.current_state['gov_score'] * 0.34
        )

        # Financial impact
        self.current_state['financial_impact'] -= action['cost']

        # Increment step counter
        self.current_state['step'] += 1

        # Record the action
        self.current_state['actions_taken'].append({
            'id': action_id,
            'name': action['name'],
            'improvement': improvement,
            'pillar': pillar,
            'cost': action['cost']
        })

        return self.current_state

# Example company data
company_data = {
    'name': 'ACS Actividades de Construccion y Servicios SA',
    'initial_scores': {'overall': 2.63, 'env': 1.11, 'social': 4.20, 'gov': 3.64},
    'available_actions': [
        {'id': 0, 'name': 'Supply Chain Emissions Program', 'pillar': 'environmental', 'cost': 59.50, 'base_improvement': 0.74},
        {'id': 1, 'name': 'Board Independence Enhancement', 'pillar': 'governance', 'cost': 24.50, 'base_improvement': 0.20},
        {'id': 2, 'name': 'Integrated Sustainability Strategy', 'pillar': 'all', 'cost': 70.00, 'base_improvement': 0.31},
        {'id': 3, 'name': 'Renewable Energy Investment', 'pillar': 'environmental', 'cost': 63.00, 'base_improvement': 0.78},
        {'id': 4, 'name': 'Diversity & Inclusion Program', 'pillar': 'social', 'cost': 50.00, 'base_improvement': 0.45},
        {'id': 5, 'name': 'Employee Wellbeing Initiative', 'pillar': 'social', 'cost': 45.00, 'base_improvement': 0.40},
        {'id': 6, 'name': 'Ethics & Compliance Program', 'pillar': 'governance', 'cost': 38.00, 'base_improvement': 0.25},
        {'id': 7, 'name': 'Water Conservation Program', 'pillar': 'environmental', 'cost': 70.00, 'base_improvement': 0.60},
        {'id': 8, 'name': 'Energy Efficiency Upgrade', 'pillar': 'environmental', 'cost': 80.00, 'base_improvement': 0.70},
        {'id': 9, 'name': 'Community Engagement Project', 'pillar': 'social', 'cost': 60.00, 'base_improvement': 0.50}
    ]
}

# Create the environment
env = SimplifiedESGEnvironment(company_data)

# Create an interactive simulator
def create_interactive_simulator():
    # Initialize state
    state = env.reset()
    max_steps = 8

    # Create action selection dropdown
    action_options = [(f"{a['id']}: {a['name']} (${a['cost']:.2f}) - {a['pillar']}", a['id']) for a in company_data['available_actions']]
    action_dropdown = widgets.Dropdown(
        options=action_options,
        description='Select Action:',
        style={'description_width': 'initial'},
        layout={'width': '80%', 'margin': '10px 0'}
    )

    # Create action button
    action_button = widgets.Button(
        description='Take Action',
        button_style='success',
        layout={'width': '200px', 'margin': '10px 0'}
    )

    # Create reset button
    reset_button = widgets.Button(
        description='Reset Simulation',
        button_style='danger',
        layout={'width': '200px', 'margin': '10px 0'}
    )

    # Create optimal strategy button
    optimal_button = widgets.Button(
        description='Show Optimal Strategy',
        button_style='info',
        layout={'width': '200px', 'margin': '10px 0'}
    )

    # Create output area
    output = widgets.Output()

    # Define button callbacks
    def on_action_button_clicked(b):
        nonlocal state

        with output:
            clear_output()

            if state['step'] >= max_steps:
                print(f"Maximum steps ({max_steps}) reached. Please reset the simulation to start over.")
                display_current_state(state)
                return

            # Take the selected action
            action_id = action_dropdown.value
            state = env.step(action_id)

            print(f"Step {state['step']}: Applied {company_data['available_actions'][action_id]['name']}")
            display_current_state(state)

    def on_reset_button_clicked(b):
        nonlocal state

        with output:
            clear_output()
            state = env.reset()
            print("Simulation reset to initial state.")
            display_current_state(state)

    def on_optimal_button_clicked(b):
        with output:
            clear_output()
            print("Recommended optimal strategy based on our reinforcement learning model:")
            print("\nSequential Actions:")

            # This would be your actual model's recommendations
            # For now, using example sequence from your results
            optimal_actions = [
                {'name': 'Supply Chain Emissions Program', 'improvement': 0.74, 'cost': 59.50},
                {'name': 'Renewable Energy Investment', 'improvement': 0.98, 'cost': 63.00},
                {'name': 'Integrated Sustainability Strategy', 'improvement': 0.31, 'cost': 70.00},
                {'name': 'Board Independence Enhancement', 'improvement': 0.10, 'cost': 24.50},
                {'name': 'Supply Chain Emissions Program', 'improvement': 0.16, 'cost': 41.65},
                {'name': 'Integrated Sustainability Strategy', 'improvement': 0.22, 'cost': 49.00},
                {'name': 'Renewable Energy Investment', 'improvement': 0.16, 'cost': 44.10},
                {'name': 'Board Independence Enhancement', 'improvement': 0.07, 'cost': 17.15}
            ]

            for i, action in enumerate(optimal_actions):
                print(f"{i+1}. {action['name']} - Improvement: {action['improvement']:.2f}, Cost: ${action['cost']:.2f}")

            print("\nExpected Results:")
            print(f"Initial ESG Score: {company_data['initial_scores']['overall']:.2f}")
            print(f"Final ESG Score: 5.56 (Improvement: 2.93 points)")
            print(f"Total Investment: $369.90")
            print(f"ROI: 0.0079 ESG points per dollar")

            # Create a comparison visualization
            fig, ax = plt.subplots(figsize=(12, 6))

            # Before-after comparison
            initial_values = [company_data['initial_scores']['overall'],
                             company_data['initial_scores']['env'],
                             company_data['initial_scores']['social'],
                             company_data['initial_scores']['gov']]
            final_values = [5.56, 4.80, 5.42, 5.28]  # Example values

            x = np.arange(4)
            labels = ['Overall ESG', 'Environmental', 'Social', 'Governance']
            width = 0.35

            ax.bar(x - width/2, initial_values, width, label='Initial', color='#cccccc')
            ax.bar(x + width/2, final_values, width, label='Optimized',
                   color=['#32a852', '#4CAF50', '#2196F3', '#9C27B0'])

            ax.set_title('Optimized ESG Strategy Impact', fontsize=16)
            ax.set_xticks(x)
            ax.set_xticklabels(labels, fontsize=12)
            ax.set_ylabel('Score (0-10)', fontsize=12)
            ax.set_ylim(0, 10)
            ax.legend()

            for i, v in enumerate(initial_values):
                ax.text(i - width/2, v + 0.1, f"{v:.1f}", ha='center')

            for i, v in enumerate(final_values):
                ax.text(i + width/2, v + 0.1, f"{v:.1f}", ha='center')

            plt.tight_layout()
            plt.show()

    # Attach callbacks
    action_button.on_click(on_action_button_clicked)
    reset_button.on_click(on_reset_button_clicked)
    optimal_button.on_click(on_optimal_button_clicked)

    # Helper function to display the current state
    def display_current_state(state):
        # Display state information
        print(f"\nCurrent ESG Score: {state['esg_score']:.2f}")
        print(f"Environmental Score: {state['env_score']:.2f}")
        print(f"Social Score: {state['social_score']:.2f}")
        print(f"Governance Score: {state['gov_score']:.2f}")
        print(f"Financial Impact: ${state['financial_impact']:.2f}")
        print(f"Steps taken: {state['step']}/{max_steps}")

        if state['step'] > 0:
            print("\nActions taken:")
            for i, action in enumerate(state['actions_taken']):
                print(f"{i+1}. {action['name']} (Pillar: {action['pillar']}) - Improvement: {action['improvement']:.2f}")

        # Visualize current state
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

        # ESG score progress
        initial_score = company_data['initial_scores']['overall']
        scores = [initial_score] + [initial_score + sum(a['improvement'] for a in state['actions_taken'][:i+1])
                                   for i in range(len(state['actions_taken']))]
        steps = list(range(len(scores)))

        ax1.plot(steps, scores, marker='o', linestyle='-', linewidth=2, color='#4CAF50')
        ax1.set_title('ESG Score Progress', fontsize=16)
        ax1.set_xlabel('Step', fontsize=12)
        ax1.set_ylabel('ESG Score', fontsize=12)
        ax1.set_ylim(0, 10)
        ax1.grid(True, alpha=0.3)

        # Pillar comparison
        labels = ['Environmental', 'Social', 'Governance']
        initial_values = [company_data['initial_scores']['env'],
                         company_data['initial_scores']['social'],
                         company_data['initial_scores']['gov']]
        current_values = [state['env_score'], state['social_score'], state['gov_score']]

        x = np.arange(len(labels))
        width = 0.35

        ax2.bar(x - width/2, initial_values, width, label='Initial', color='#cccccc')
        ax2.bar(x + width/2, current_values, width, label='Current',
               color=['#4CAF50', '#2196F3', '#9C27B0'])

        ax2.set_title('ESG Pillar Comparison', fontsize=16)
        ax2.set_xticks(x)
        ax2.set_xticklabels(labels, fontsize=12)
        ax2.set_ylabel('Score (0-10)', fontsize=12)
        ax2.set_ylim(0, 10)
        ax2.legend()

        plt.tight_layout()
        plt.show()

    # Display initial state
    with output:
        print("Welcome to the ESG Strategy Simulator!")
        print(f"Company: {company_data['name']}")
        print("\nSelect actions to optimize the company's ESG performance.")
        print(f"You can take up to {max_steps} actions.")
        display_current_state(state)

    # Layout the UI elements
    display(widgets.VBox([
        widgets.HTML("<h2>ESG Strategy Simulator</h2>"),
        widgets.HBox([action_dropdown]),
        widgets.HBox([action_button, reset_button, optimal_button]),
        output
    ]))

# Run the simulator
create_interactive_simulator()

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Load sample data
def load_sample_data():
    # Create sample industry benchmark data
    industries = ['Energy', 'Utilities', 'Materials', 'Industrials']

    benchmarks = {}
    for industry in industries:
        # Random benchmark values for illustration
        benchmarks[industry] = {
            'BESG ESG Score': {
                'p25': np.random.uniform(1.5, 2.5),
                'p50': np.random.uniform(3.0, 4.0),
                'p75': np.random.uniform(5.0, 6.0),
                'p90': np.random.uniform(6.5, 7.5),
                'mean': np.random.uniform(3.5, 4.5),
                'std': np.random.uniform(0.8, 1.2)
            },
            'BESG Environmental Pillar Score': {
                'p25': np.random.uniform(1.0, 2.0),
                'p50': np.random.uniform(2.5, 3.5),
                'p75': np.random.uniform(4.5, 5.5),
                'p90': np.random.uniform(6.0, 7.0),
                'mean': np.random.uniform(3.0, 4.0),
                'std': np.random.uniform(0.8, 1.2)
            },
            'BESG Social Pillar Score': {
                'p25': np.random.uniform(1.5, 2.5),
                'p50': np.random.uniform(3.0, 4.0),
                'p75': np.random.uniform(5.0, 6.0),
                'p90': np.random.uniform(6.5, 7.5),
                'mean': np.random.uniform(3.5, 4.5),
                'std': np.random.uniform(0.8, 1.2)
            },
            'BESG Governance Pillar Score': {
                'p25': np.random.uniform(2.0, 3.0),
                'p50': np.random.uniform(3.5, 4.5),
                'p75': np.random.uniform(5.5, 6.5),
                'p90': np.random.uniform(7.0, 8.0),
                'mean': np.random.uniform(4.0, 5.0),
                'std': np.random.uniform(0.8, 1.2)
            }
        }

    # Create sample company data
    companies = {
        'ACS Actividades de Construccion y Servicios SA': {
            'industry': 'Industrials',
            'initial': {
                'BESG ESG Score': 2.63,
                'BESG Environmental Pillar Score': 1.11,
                'BESG Social Pillar Score': 4.20,
                'BESG Governance Pillar Score': 3.64
            },
            'optimized': {
                'BESG ESG Score': 5.56,
                'BESG Environmental Pillar Score': 4.80,
                'BESG Social Pillar Score': 5.42,
                'BESG Governance Pillar Score': 5.28
            }
        },
        'AKITA Drilling Ltd': {
            'industry': 'Energy',
            'initial': {
                'BESG ESG Score': 1.85,
                'BESG Environmental Pillar Score': 1.20,
                'BESG Social Pillar Score': 2.30,
                'BESG Governance Pillar Score': 2.50
            },
            'optimized': {
                'BESG ESG Score': 4.75,
                'BESG Environmental Pillar Score': 4.12,
                'BESG Social Pillar Score': 4.89,
                'BESG Governance Pillar Score': 5.10
            }
        },
        'ARC Resources Ltd': {
            'industry': 'Energy',
            'initial': {
                'BESG ESG Score': 3.20,
                'BESG Environmental Pillar Score': 2.75,
                'BESG Social Pillar Score': 3.15,
                'BESG Governance Pillar Score': 4.10
            },
            'optimized': {
                'BESG ESG Score': 6.10,
                'BESG Environmental Pillar Score': 5.85,
                'BESG Social Pillar Score': 5.90,
                'BESG Governance Pillar Score': 6.40
            }
        },
        'Sapura Energy Bhd': {
            'industry': 'Energy',
            'initial': {
                'BESG ESG Score': 2.40,
                'BESG Environmental Pillar Score': 1.95,
                'BESG Social Pillar Score': 2.60,
                'BESG Governance Pillar Score': 3.10
            },
            'optimized': {
                'BESG ESG Score': 5.20,
                'BESG Environmental Pillar Score': 4.65,
                'BESG Social Pillar Score': 5.30,
                'BESG Governance Pillar Score': 5.50
            }
        },
        'Snam SpA': {
            'industry': 'Utilities',
            'initial': {
                'BESG ESG Score': 4.10,
                'BESG Environmental Pillar Score': 3.80,
                'BESG Social Pillar Score': 4.25,
                'BESG Governance Pillar Score': 4.50
            },
            'optimized': {
                'BESG ESG Score': 7.20,
                'BESG Environmental Pillar Score': 7.10,
                'BESG Social Pillar Score': 7.05,
                'BESG Governance Pillar Score': 7.30
            }
        }
    }

    return benchmarks, companies

# Load the data
benchmarks, companies = load_sample_data()

# Create interactive benchmark comparison
def create_benchmark_comparison():
    # Create company selector
    company_options = list(companies.keys())
    company_dropdown = widgets.Dropdown(
        options=company_options,
        value=company_options[0],
        description='Company:',
        style={'description_width': 'initial'},
        layout={'width': '50%', 'margin': '10px 0'}
    )

    # Create metric selector
    metric_options = ['BESG ESG Score', 'BESG Environmental Pillar Score',
                     'BESG Social Pillar Score', 'BESG Governance Pillar Score']
    metric_dropdown = widgets.Dropdown(
        options=metric_options,
        value=metric_options[0],
        description='Metric:',
        style={'description_width': 'initial'},
        layout={'width': '50%', 'margin': '10px 0'}
    )

    # Create toggle for initial vs. optimized view
    view_toggle = widgets.ToggleButtons(
        options=['Initial vs. Industry', 'Optimized vs. Industry', 'Before & After'],
        value='Initial vs. Industry',
        description='View:',
        button_style='info',
        style={'description_width': 'initial'},
        layout={'width': '70%', 'margin': '10px 0'}
    )

    # Create output area
    output = widgets.Output()

    # Update function
    def update_visualization(company, metric, view):
        with output:
            clear_output()

            # Get company data
            company_data = companies[company]
            industry = company_data['industry']
            industry_benchmarks = benchmarks[industry][metric]

            # Create visualization based on view
            if view == 'Initial vs. Industry':
                plot_company_vs_industry(company, company_data['initial'][metric], metric, industry, industry_benchmarks)
            elif view == 'Optimized vs. Industry':
                plot_company_vs_industry(company, company_data['optimized'][metric], metric, industry, industry_benchmarks)
            else:  # Before & After
                plot_before_after(company, company_data['initial'][metric], company_data['optimized'][metric],
                                 metric, industry, industry_benchmarks)

            # Add benchmark table
            display_benchmark_table(company_data, industry_benchmarks, metric, view)

    # Helper function to plot company vs industry
    def plot_company_vs_industry(company, company_value, metric, industry, industry_benchmarks):
        fig, ax = plt.subplots(figsize=(12, 6))

        # Create data for visualization
        positions = [1, 2, 3, 4, 5]
        labels = ['Bottom 25%', '25-50%', '50-75%', '75-90%', 'Top 10%']
        values = [
            industry_benchmarks['p25'],
            industry_benchmarks['p50'] - industry_benchmarks['p25'],
            industry_benchmarks['p75'] - industry_benchmarks['p50'],
            industry_benchmarks['p90'] - industry_benchmarks['p75'],
            10 - industry_benchmarks['p90']  # Assuming 10 is max score
        ]

        # Create stacked bar for percentile ranges
        colors = ['#f44336', '#ff9800', '#ffc107', '#8bc34a', '#4caf50']
        bottom = 0
        for i, (value, color) in enumerate(zip(values, colors)):
            ax.bar(1, value, bottom=bottom, color=color, alpha=0.7, width=0.5)

            # Add text for range boundaries
            if i == 0:
                ax.text(1.05, bottom + value/2, f"{labels[i]}", ha='left', va='center')
                ax.text(0.95, bottom + value/2, f"{0:.1f}-{industry_benchmarks['p25']:.1f}", ha='right', va='center')
            elif i == 4:
                ax.text(1.05, bottom + value/2, f"{labels[i]}", ha='left', va='center')
                ax.text(0.95, bottom + value/2, f"{industry_benchmarks['p90']:.1f}+", ha='right', va='center')
            else:
                ax.text(1.05, bottom + value/2, f"{labels[i]}", ha='left', va='center')

                # Get the range boundaries
                lower = [0, industry_benchmarks['p25'], industry_benchmarks['p50'],
                        industry_benchmarks['p75'], industry_benchmarks['p90']][i]
                upper = [industry_benchmarks['p25'], industry_benchmarks['p50'],
                         industry_benchmarks['p75'], industry_benchmarks['p90'], 10][i]

                ax.text(0.95, bottom + value/2, f"{lower:.1f}-{upper:.1f}", ha='right', va='center')

            bottom += value

        # Add marker for company value
        ax.plot([0.7, 1.3], [company_value, company_value], 'k-', lw=2)
        ax.plot(1, company_value, 'ko', ms=10)

        # Add text for company value
        ax.text(1.3, company_value, f"{company}\n{company_value:.2f}", ha='left', va='center',
                fontweight='bold', bbox=dict(facecolor='white', alpha=0.7))

        # Configure the plot
        ax.set_xlim(0.5, 3)
        ax.set_ylim(0, 10)
        ax.set_title(f"{metric} - {company} vs. {industry} Industry", fontsize=16)
        ax.set_ylabel('Score (0-10)', fontsize=14)
        ax.set_xticks([])

        plt.tight_layout()
        plt.show()

    # Helper function to plot before and after
    def plot_before_after(company, initial_value, optimized_value, metric, industry, industry_benchmarks):
        fig, ax = plt.subplots(figsize=(12, 6))

        # Create data for visualization
        positions = [1, 2]
        labels = ['Initial', 'Optimized']
        values = [initial_value, optimized_value]

        # Plot bars
        bars = ax.bar(positions, values, color=['#cccccc', '#4caf50'], width=0.6)

        # Add text labels
        for i, v in enumerate(values):
            ax.text(positions[i], v + 0.2, f"{v:.2f}", ha='center', fontsize=12)

        # Add industry benchmark lines
        benchmarks_to_plot = [
            ('Bottom 25%', industry_benchmarks['p25'], '#f44336'),
            ('Median', industry_benchmarks['p50'], '#ff9800'),
            ('Top 25%', industry_benchmarks['p75'], '#8bc34a'),
            ('Top 10%', industry_benchmarks['p90'], '#4caf50')
        ]

        for label, value, color in benchmarks_to_plot:
            ax.axhline(y=value, linestyle='--', color=color, alpha=0.7)
            ax.text(2.2, value, f"{label}: {value:.2f}", ha='left', va='center',
                    color=color, fontweight='bold')

        # Configure the plot
        ax.set_title(f"{metric} - {company} Before & After Optimization", fontsize=16)
        ax.set_ylabel('Score (0-10)', fontsize=14)
        ax.set_ylim(0, 10)
        ax.set_xticks(positions)
        ax.set_xticklabels(labels, fontsize=12)

        # Highlight the improvement
        improvement = optimized_value - initial_value
        ax.annotate('', xy=(2, initial_value), xytext=(2, optimized_value),
                   arrowprops=dict(arrowstyle='<->', color='blue', lw=2))
        ax.text(2.2, initial_value + improvement/2, f"Improvement: +{improvement:.2f}",
                color='blue', fontweight='bold', va='center')

        # Identify the percentile changes
        initial_percentile = get_percentile_category(initial_value, industry_benchmarks)
        optimized_percentile = get_percentile_category(optimized_value, industry_benchmarks)

        if initial_percentile != optimized_percentile:
            ax.text(1.5, 9.5, f"Percentile Improvement: {initial_percentile} → {optimized_percentile}",
                    ha='center', fontweight='bold', bbox=dict(facecolor='lightyellow', alpha=0.9))

        plt.tight_layout()
        plt.show()

    # Helper function to determine percentile category
    def get_percentile_category(value, benchmarks):
        if value < benchmarks['p25']:
            return "Bottom 25%"
        elif value < benchmarks['p50']:
            return "25-50%"
        elif value < benchmarks['p75']:
            return "50-75%"
        elif value < benchmarks['p90']:
            return "75-90%"
        else:
            return "Top 10%"

    # Helper function to display benchmark table
    def display_benchmark_table(company_data, industry_benchmarks, metric, view):
        # Get company values
        initial_value = company_data['initial'][metric]
        optimized_value = company_data['optimized'][metric]

        # Determine percentiles
        initial_percentile = get_percentile_category(initial_value, industry_benchmarks)
        optimized_percentile = get_percentile_category(optimized_value, industry_benchmarks)

        # Create HTML table
        html = f"""
        <div style="background-color:#f5f5f5; padding:15px; border-radius:10px; margin-top:20px;">
            <h3 style="color:#333;">Industry Benchmark Comparison: {metric}</h3>
            <table style="width:100%; border-collapse:collapse; margin-top:10px;">
                <tr style="background-color:#e0e0e0;">
                    <th style="padding:8px; text-align:left; border:1px solid #ddd;">Benchmark</th>
                    <th style="padding:8px; text-align:center; border:1px solid #ddd;">Value</th>
                    <th style="padding:8px; text-align:center; border:1px solid #ddd;">Initial Position</th>
                    <th style="padding:8px; text-align:center; border:1px solid #ddd;">Optimized Position</th>
                </tr>
                <tr>
                    <td style="padding:8px; border:1px solid #ddd;">Bottom 25% Threshold</td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;">{industry_benchmarks['p25']:.2f}</td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;
                        {get_cell_style(initial_value, industry_benchmarks['p25'], '<')}">
                        {get_comparison_text(initial_value, industry_benchmarks['p25'], '<')}
                    </td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;
                        {get_cell_style(optimized_value, industry_benchmarks['p25'], '<')}">
                        {get_comparison_text(optimized_value, industry_benchmarks['p25'], '<')}
                    </td>
                </tr>
                <tr>
                    <td style="padding:8px; border:1px solid #ddd;">Industry Median (50%)</td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;">{industry_benchmarks['p50']:.2f}</td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;
                        {get_cell_style(initial_value, industry_benchmarks['p50'], '<')}">
                        {get_comparison_text(initial_value, industry_benchmarks['p50'], '<')}
                    </td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;
                        {get_cell_style(optimized_value, industry_benchmarks['p50'], '<')}">
                        {get_comparison_text(optimized_value, industry_benchmarks['p50'], '<')}
                    </td>
                </tr>
                <tr>
                    <td style="padding:8px; border:1px solid #ddd;">Top 25% Threshold</td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;">{industry_benchmarks['p75']:.2f}</td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;
                        {get_cell_style(initial_value, industry_benchmarks['p75'], '>')}">
                        {get_comparison_text(initial_value, industry_benchmarks['p75'], '>')}
                    </td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;
                        {get_cell_style(optimized_value, industry_benchmarks['p75'], '>')}">
                        {get_comparison_text(optimized_value, industry_benchmarks['p75'], '>')}
                    </td>
                </tr>
                <tr>
                    <td style="padding:8px; border:1px solid #ddd;">Top 10% Threshold</td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;">{industry_benchmarks['p90']:.2f}</td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;
                        {get_cell_style(initial_value, industry_benchmarks['p90'], '>')}">
                        {get_comparison_text(initial_value, industry_benchmarks['p90'], '>')}
                    </td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;
                        {get_cell_style(optimized_value, industry_benchmarks['p90'], '>')}">
                        {get_comparison_text(optimized_value, industry_benchmarks['p90'], '>')}
                    </td>
                </tr>
                <tr style="background-color:#e0e0e0; font-weight:bold;">
                    <td style="padding:8px; border:1px solid #ddd;">Company Score</td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;">-</td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;">{initial_value:.2f} ({initial_percentile})</td>
                    <td style="padding:8px; text-align:center; border:1px solid #ddd;">{optimized_value:.2f} ({optimized_percentile})</td>
                </tr>
            </table>
        </div>
        """

        display(HTML(html))

    # Helper function for table cell styling
    def get_cell_style(value, benchmark, comparison):
        if comparison == '<':
            return "background-color:#f8d7da;" if value < benchmark else "background-color:#d4edda;"
        else:  # '>'
            return "background-color:#d4edda;" if value > benchmark else "background-color:#f8d7da;"

    # Helper function for comparison text
    def get_comparison_text(value, benchmark, comparison):
        if comparison == '<':
            return "✓ Above" if value >= benchmark else "✗ Below"
        else:  # '>'
            return "✓ Above" if value > benchmark else "✗ Below"

    # Connect the widgets
    def on_change(change):
        update_visualization(company_dropdown.value, metric_dropdown.value, view_toggle.value)

    company_dropdown.observe(on_change, names='value')
    metric_dropdown.observe(on_change, names='value')
    view_toggle.observe(on_change, names='value')

    # Layout the UI elements
    display(widgets.VBox([
        widgets.HTML("<h2>ESG Industry Benchmark Comparison</h2>"),
        widgets.HBox([company_dropdown, metric_dropdown]),
        view_toggle,
        output
    ]))

    # Initial update
    update_visualization(company_dropdown.value, metric_dropdown.value, view_toggle.value)

# Run the benchmark comparison
create_benchmark_comparison()